## Prediction with Trained ANN Model

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import numpy as np 
from tensorflow.keras.models import load_model

In [27]:
## Load the pickle files,trained model and scaler

model= load_model('churn_model.h5')

## Load the encoder and scaler
with open('one_hot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)
    
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [28]:
## Example input data
input_data = {
    'CreditScore': 650,
    'Geography': 'France', ## Converting to numerical using label encoder
    'Gender':'Male',    ## Converting to numerical using label encoder
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [29]:
## One-hot encode the categorical features

geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()

geo_encoded_df= pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\0 ML BootCamp\52_End_to_End_Deep_Learning_Project_Using_ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [30]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,Male,40,3,60000,2,1,1,50000


In [31]:
## Encode categorical features using label encoder
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,1,40,3,60000,2,1,1,50000


In [32]:
## concatate one-hot encoded geography with the input data
input_df = pd.concat([input_df.drop('Geography',axis=1), geo_encoded_df], axis=1)
input_df

# input_data=pd.concat([input_data.reset_index(drop=True), geo_encoded_df], axis=1)
# input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [33]:
## Scale the input data
input_df_scaled = scaler.transform(input_df)
input_df_scaled

array([[-0.01709861,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [34]:
## Predict using the trained model
predicted_churn = model.predict(input_df_scaled)
predicted_churn

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step


array([[0.02972971]], dtype=float32)

In [39]:
predtiction_proba=predicted_churn[0][0]
if predtiction_proba > 0.5:
    print("There is a {:.2f}% chance that the customer will stop using our service.".format(predtiction_proba * 100)) # Yes, churn
else:
    print("There’s a {:.2f}% chance the customer will continue using our service.".format((1-predtiction_proba) * 100)) # No, not churn

There’s a 97.03% chance the customer will continue using our service.
